<a href="https://colab.research.google.com/github/tylerlum/ufc_automated_scoring_system/blob/main/UFC_Automated_Scoring_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UFC Automated Scoring System

The goal of this notebook is to:
* Explore the FightMetrics webpage to scrape the fight and fighter information we need
* Store the fight and fighter data into csv files
* Preprocess the data
* Train and evaluate a neural network to predict fight outcomes

## All bouts and all fighters

In [2]:
import pandas as pd

In [3]:
all_past_events_url = "http://ufcstats.com/statistics/events/completed?page=all"
all_past_events_tables = pd.read_html(all_past_events_url)[0] # Returns list of all tables on page

In [4]:
all_past_events_tables.head()

,Name/date,Location
0,NaN,NaN
1,"UFC Fight Night: Hall vs. Silva October 31, 2020","Las Vegas, Nevada, USA"
2,"UFC 254: Khabib vs. Gaethje October 24, 2020","Abu Dhabi, Abu Dhabi, United Arab Emirates"
3,UFC Fight Night: Ortega vs. The Korean Zombie ...,"Abu Dhabi, Abu Dhabi, United Arab Emirates"
4,UFC Fight Night: Moraes vs. Sandhagen October...,"Abu Dhabi, Abu Dhabi, United Arab Emirates"


In [5]:
from string import ascii_lowercase

all_fighters_tables = []
for c in ascii_lowercase:
  all_fighters_url = f"http://ufcstats.com/statistics/fighters?char={c}&page=all"
  all_fighters_table = pd.read_html(all_fighters_url)[0]
  all_fighters_tables.append(all_fighters_table)

all_fighters_tables = pd.concat(all_fighters_tables)

In [6]:
all_fighters_tables.head()

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tom,Aaron,NaN,--,155 lbs.,--,NaN,5.0,3.0,0.0,NaN
2,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4.0,6.0,0.0,NaN
3,David,Abbott,Tank,"6' 0""",265 lbs.,--,Switch,10.0,15.0,0.0,NaN
4,Shamil,Abdurakhimov,Abrek,"6' 3""",235 lbs.,"76.0""",Orthodox,20.0,5.0,0.0,NaN


## Getting the data for one fight

This will look at Brian Ortega vs. Chan Sung Jung.

UFC Fight Night: Ortega vs. The Korean Zombie

http://ufcstats.com/fight-details/5a440bdee19efaab

In [7]:
ortega_vs_jung_url = "http://ufcstats.com/fight-details/5a440bdee19efaab"
ortega_vs_jung_tables = pd.read_html(ortega_vs_jung_url)
print(f"This URL has {len(ortega_vs_jung_tables)} tables")

This URL has 4 tables


In [8]:
ortega_vs_jung_tables[0].head()

,Fighter,KD,Sig. str.,Sig. str. %,Total str.,Td,Td %,Sub. att,Rev.,Ctrl
0,Brian Ortega Chan Sung Jung,2 0,127 of 212 62 of 163,59% 38%,129 of 214 64 of 166,3 of 10 0 of 0,30% ---,0 0,0 0,0:57 0:00


In [9]:
ortega_vs_jung_tables[1].head()

Fighter      KD  ...    Rev.        Ctrl
                       Round 1 Round 1  ... Round 1     Round 1
                       Round 2 Round 2  ... Round 2     Round 2
                       Round 3 Round 3  ... Round 3     Round 3
                       Round 4 Round 4  ... Round 4     Round 4
                       Round 5 Round 5  ... Round 5     Round 5
0  Brian Ortega Chan Sung Jung    1  0  ...    0  0  0:00  0:00
1  Brian Ortega Chan Sung Jung    1  0  ...    0  0  0:35  0:00
2  Brian Ortega Chan Sung Jung    0  0  ...    0  0  0:00  0:00
3  Brian Ortega Chan Sung Jung    0  0  ...    0  0  0:22  0:00
4  Brian Ortega Chan Sung Jung    0  0  ...    0  0  0:00  0:00

[5 rows x 10 columns]

In [10]:
ortega_vs_jung_tables[2].head()

,Fighter,Sig. str,Sig. str. %,Head,Body,Leg,Distance,Clinch,Ground
0,Brian Ortega Chan Sung Jung,127 of 212 62 of 163,59% 38%,64 of 135 35 of 127,22 of 28 14 of 21,41 of 49 13 of 15,118 of 199 62 of 162,5 of 7 0 of 1,4 of 6 0 of 0


In [11]:
ortega_vs_jung_tables[3].head()

,Fighter,Sig. str,Sig. str. %,Head,Body,Leg,Distance,Clinch,Ground,Unnamed: 9_level_0
,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1
,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2
,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3
,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4
,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5
0,Brian Ortega Chan Sung Jung,24 of 39 14 of 26,61% 53%,11 of 24 4 of 15,5 of 6 4 of 5,8 of 9 6 of 6,24 of 39 14 of 26,0 of 0 0 of 0,0 of 0 0 of 0,NaN
1,Brian Ortega Chan Sung Jung,23 of 37 14 of 34,62% 41%,14 of 25 12 of 31,2 of 4 0 of 1,7 of 8 2 of 2,17 of 29 14 of 34,2 of 2 0 of 0,4 of 6 0 of 0,NaN
2,Brian Ortega Chan Sung Jung,28 of 42 7 of 32,66% 21%,13 of 25 2 of 23,4 of 4 1 of 3,11 of 13 4 of 6,27 of 41 7 of 32,1 of 1 0 of 0,0 of 0 0 of 0,NaN
3,Brian Ortega Chan Sung Jung,17 of 35 8 of 26,48% 30%,5 of 18 1 of 18,5 of 6 6 of 7,7 of 11 1 of 1,15 of 33 8 of 26,2 of 2 0 of 0,0 of 0 0 of 0,NaN
4,Brian Ortega Chan Sung Jung,35 of 59 19 of 45,59% 42%,21 of 43 16 of 40,6 of 8 3 of 5,8 of 8 0 of 0,35 of 57 19 of 44,0 of 2 0 of 1,0 of 0 0 of 0,NaN


Note how each cell actually has two datapoints, which is a bit frustrating. We need to break them up into different cells.

In [12]:
ortega_vs_jung_tables[0].head()

,Fighter,KD,Sig. str.,Sig. str. %,Total str.,Td,Td %,Sub. att,Rev.,Ctrl
0,Brian Ortega Chan Sung Jung,2 0,127 of 212 62 of 163,59% 38%,129 of 214 64 of 166,3 of 10 0 of 0,30% ---,0 0,0 0,0:57 0:00


In [13]:
ortega_vs_jung_tables[0]["Fighter"][0]

'Brian Ortega Chan Sung Jung'

In [14]:
for i, row in all_fighters_tables.iterrows():
    fighter_name = f'{row["First"]} {row["Last"]}'
    if ortega_vs_jung_tables[0]["Fighter"][0].startswith(fighter_name):
        first_fighter = fighter_name
        second_fighter = ortega_vs_jung_tables[0]["Fighter"][0][len(fighter_name)+1:]
        print(f"First fighter is {first_fighter}")
        print(f'Second fighter is {second_fighter}')
        break

First fighter is Brian Ortega
Second fighter is Chan Sung Jung


In [17]:
def parse_two_fighter_stats(fight_table, column):
    string_split = fight_table[column][0].split(" ")
    first_fighter_stat = " ".join(string_split[:len(string_split)//2])
    second_fighter_stat = " ".join(string_split[len(string_split)//2+1:])
    return first_fighter_stat, second_fighter_stat

In [18]:
for column in ortega_vs_jung_tables[0].columns:
    if column == "Fighter":
        continue
    fighter_1_stat, fighter_2_stat = parse_two_fighter_stats(ortega_vs_jung_tables[0], column)
    print(f"{column} 1: {fighter_1_stat}")
    print(f"{column} 2: {fighter_2_stat}")
    print()

KD 1: 2
KD 2: 0

Sig. str. 1: 127 of 212
Sig. str. 2: 62 of 163

Sig. str. % 1: 59%
Sig. str. % 2: 38%

Total str. 1: 129 of 214
Total str. 2: 64 of 166

Td 1: 3 of 10
Td 2: 0 of 0

Td % 1: 30%
Td % 2: ---

Sub. att 1: 0
Sub. att 2: 0

Rev. 1: 0
Rev. 2: 0

Ctrl 1: 0:57
Ctrl 2: 0:00



Now do the same for other tables

In [19]:
ortega_vs_jung_tables[1]

Fighter      KD  ...    Rev.        Ctrl
                       Round 1 Round 1  ... Round 1     Round 1
                       Round 2 Round 2  ... Round 2     Round 2
                       Round 3 Round 3  ... Round 3     Round 3
                       Round 4 Round 4  ... Round 4     Round 4
                       Round 5 Round 5  ... Round 5     Round 5
0  Brian Ortega Chan Sung Jung    1  0  ...    0  0  0:00  0:00
1  Brian Ortega Chan Sung Jung    1  0  ...    0  0  0:35  0:00
2  Brian Ortega Chan Sung Jung    0  0  ...    0  0  0:00  0:00
3  Brian Ortega Chan Sung Jung    0  0  ...    0  0  0:22  0:00
4  Brian Ortega Chan Sung Jung    0  0  ...    0  0  0:00  0:00

[5 rows x 10 columns]

In [78]:
new_columns = ["Fighter 1", "Fighter 2"]
for i, column in enumerate(ortega_vs_jung_tables[1]):
    if column[0] == "Fighter":
        continue
    new_columns.append(f"Fighter 1 {column[0]}")
    new_columns.append(f"Fighter 2 {column[0]}")
print(new_columns)

['Fighter 1', 'Fighter 2', 'Fighter 1 KD', 'Fighter 2 KD', 'Fighter 1 Sig. str.', 'Fighter 2 Sig. str.', 'Fighter 1 Sig. str. %', 'Fighter 2 Sig. str. %', 'Fighter 1 Total str.', 'Fighter 2 Total str.', 'Fighter 1 Td %', 'Fighter 2 Td %', 'Fighter 1 Td %', 'Fighter 2 Td %', 'Fighter 1 Sub. att', 'Fighter 2 Sub. att', 'Fighter 1 Rev.', 'Fighter 2 Rev.', 'Fighter 1 Ctrl', 'Fighter 2 Ctrl']


In [21]:
def parse_string(row_string):
    string_split = row_string.split(" ")
    first_fighter_stat = " ".join(string_split[:len(string_split)//2])
    second_fighter_stat = " ".join(string_split[len(string_split)//2+1:])
    return first_fighter_stat, second_fighter_stat

In [24]:
new_rows = []
for i, row in ortega_vs_jung_tables[1].iterrows():
    new_row = []
    new_row.append(first_fighter)
    new_row.append(second_fighter)
    for column in ortega_vs_jung_tables[1]:
        if column[0] == "Fighter":
            continue
        stat1, stat2 = parse_string(row[column[0]][0])
        new_row.append(stat1)
        new_row.append(stat2)
    new_rows.append(new_row)

In [25]:
print(new_rows)

[['Brian Ortega', 'Chan Sung Jung', '1', '0', '24 of 39', '14 of 26', '61%', '53%', '24 of 39', '14 of 26', '0 of 0', '0 of 0', '0 of 0', '0 of 0', '0', '0', '0', '0', '0:00', '0:00'], ['Brian Ortega', 'Chan Sung Jung', '1', '0', '23 of 37', '14 of 34', '62%', '41%', '24 of 38', '14 of 34', '2 of 3', '0 of 0', '2 of 3', '0 of 0', '0', '0', '0', '0', '0:35', '0:00'], ['Brian Ortega', 'Chan Sung Jung', '0', '0', '28 of 42', '7 of 32', '66%', '21%', '28 of 42', '7 of 32', '0 of 1', '0 of 0', '0 of 1', '0 of 0', '0', '0', '0', '0', '0:00', '0:00'], ['Brian Ortega', 'Chan Sung Jung', '0', '0', '17 of 35', '8 of 26', '48%', '30%', '18 of 36', '10 of 29', '1 of 4', '0 of 0', '1 of 4', '0 of 0', '0', '0', '0', '0', '0:22', '0:00'], ['Brian Ortega', 'Chan Sung Jung', '0', '0', '35 of 59', '19 of 45', '59%', '42%', '35 of 59', '19 of 45', '0 of 2', '0 of 0', '0 of 2', '0 of 0', '0', '0', '0', '0', '0:00', '0:00']]


In [26]:
df = pd.DataFrame(new_rows, columns=new_columns) 

In [27]:
df.head()

,Fighter 1,Fighter 2,Fighter 1 KD,Fighter 2 KD,Fighter 1 Sig. str.,Fighter 2 Sig. str.,Fighter 1 Sig. str. %,Fighter 2 Sig. str. %,Fighter 1 Total str.,Fighter 2 Total str.,Fighter 1 Td %,Fighter 2 Td %,Fighter 1 Td %,Fighter 2 Td %,Fighter 1 Sub. att,Fighter 2 Sub. att,Fighter 1 Rev.,Fighter 2 Rev.,Fighter 1 Ctrl,Fighter 2 Ctrl
0,Brian Ortega,Chan Sung Jung,1,0,24 of 39,14 of 26,61%,53%,24 of 39,14 of 26,0 of 0,0 of 0,0 of 0,0 of 0,0,0,0,0,0:00,0:00
1,Brian Ortega,Chan Sung Jung,1,0,23 of 37,14 of 34,62%,41%,24 of 38,14 of 34,2 of 3,0 of 0,2 of 3,0 of 0,0,0,0,0,0:35,0:00
2,Brian Ortega,Chan Sung Jung,0,0,28 of 42,7 of 32,66%,21%,28 of 42,7 of 32,0 of 1,0 of 0,0 of 1,0 of 0,0,0,0,0,0:00,0:00
3,Brian Ortega,Chan Sung Jung,0,0,17 of 35,8 of 26,48%,30%,18 of 36,10 of 29,1 of 4,0 of 0,1 of 4,0 of 0,0,0,0,0,0:22,0:00
4,Brian Ortega,Chan Sung Jung,0,0,35 of 59,19 of 45,59%,42%,35 of 59,19 of 45,0 of 2,0 of 0,0 of 2,0 of 0,0,0,0,0,0:00,0:00


Do the same for the other table

In [80]:
new_columns = ["Fighter 1", "Fighter 2"]
for i, column in enumerate(ortega_vs_jung_tables[3]):
    if column[0] == "Fighter" or column[0] == "Unnamed: 9_level_0":
        continue
    new_columns.append(f"Fighter 1 {column[0]}")
    new_columns.append(f"Fighter 2 {column[0]}")
print(new_columns)

['Fighter 1', 'Fighter 2', 'Fighter 1 Sig. str', 'Fighter 2 Sig. str', 'Fighter 1 Sig. str. %', 'Fighter 2 Sig. str. %', 'Fighter 1 Head', 'Fighter 2 Head', 'Fighter 1 Body', 'Fighter 2 Body', 'Fighter 1 Leg', 'Fighter 2 Leg', 'Fighter 1 Distance', 'Fighter 2 Distance', 'Fighter 1 Clinch', 'Fighter 2 Clinch', 'Fighter 1 Ground', 'Fighter 2 Ground']


In [81]:
new_rows = []
for i, row in ortega_vs_jung_tables[3].iterrows():
    new_row = []
    new_row.append(first_fighter)
    new_row.append(second_fighter)
    for column in ortega_vs_jung_tables[3]:
        if column[0] == "Fighter" or column[0] == "Unnamed: 9_level_0":
            continue
        stat1, stat2 = parse_string(row[column[0]][0])
        new_row.append(stat1)
        new_row.append(stat2)
    new_rows.append(new_row)

In [82]:
df2 = pd.DataFrame(new_rows, columns=new_columns) 

In [83]:
df2.head()

,Fighter 1,Fighter 2,Fighter 1 Sig. str,Fighter 2 Sig. str,Fighter 1 Sig. str. %,Fighter 2 Sig. str. %,Fighter 1 Head,Fighter 2 Head,Fighter 1 Body,Fighter 2 Body,Fighter 1 Leg,Fighter 2 Leg,Fighter 1 Distance,Fighter 2 Distance,Fighter 1 Clinch,Fighter 2 Clinch,Fighter 1 Ground,Fighter 2 Ground
0,Brian Ortega,Chan Sung Jung,24 of 39,14 of 26,61%,53%,11 of 24,4 of 15,5 of 6,4 of 5,8 of 9,6 of 6,24 of 39,14 of 26,0 of 0,0 of 0,0 of 0,0 of 0
1,Brian Ortega,Chan Sung Jung,23 of 37,14 of 34,62%,41%,14 of 25,12 of 31,2 of 4,0 of 1,7 of 8,2 of 2,17 of 29,14 of 34,2 of 2,0 of 0,4 of 6,0 of 0
2,Brian Ortega,Chan Sung Jung,28 of 42,7 of 32,66%,21%,13 of 25,2 of 23,4 of 4,1 of 3,11 of 13,4 of 6,27 of 41,7 of 32,1 of 1,0 of 0,0 of 0,0 of 0
3,Brian Ortega,Chan Sung Jung,17 of 35,8 of 26,48%,30%,5 of 18,1 of 18,5 of 6,6 of 7,7 of 11,1 of 1,15 of 33,8 of 26,2 of 2,0 of 0,0 of 0,0 of 0
4,Brian Ortega,Chan Sung Jung,35 of 59,19 of 45,59%,42%,21 of 43,16 of 40,6 of 8,3 of 5,8 of 8,0 of 0,35 of 57,19 of 44,0 of 2,0 of 1,0 of 0,0 of 0


## Going to future pages

In [140]:
from urllib.request import urlopen
import numpy as np
from string import ascii_uppercase
from dateutil import parser
from datetime import datetime

In [142]:
def find_latest_index_with_char(string, chars):
    for i in reversed(range(len(string))):
        if string[i] in chars:
            return i
    return -1

def get_all_events(all_past_events_url):
    all_past_events_page = urlopen(all_past_events_url)
    all_past_events_html_bytes = all_past_events_page.read()
    all_past_events_html = all_past_events_html_bytes.decode("utf-8")
    all_past_events_tables = pd.read_html(all_past_events_url)[0]
    all_past_events_tables = all_past_events_tables[all_past_events_tables["Name/date"].notna()]

    event_names = []
    for i, row in all_past_events_tables.iterrows():
        date_index = find_latest_index_with_char(row["Name/date"], ascii_uppercase)
        event_name = row["Name/date"][:date_index-1].strip()
        date = parser.parse(row["Name/date"][date_index:])
        if date < datetime.now():
            event_names.append(event_name)

    all_http_strings = []
    for event_name in event_names:
        new_substring = all_past_events_html[:all_past_events_html.index(event_name)]
        http_index = new_substring.rfind("http://")
        http_string = new_substring[http_index:]
        http_string = http_string[:http_string.find("\"")]
        all_http_strings.append(http_string)
    return all_http_strings

all_event_http_strings = get_all_events(all_past_events_url)

In [143]:
print(len(all_event_http_strings))

537


In [155]:
def get_all_fights(past_event_url):
    past_event_page = urlopen(past_event_url)
    past_event_html_bytes = past_event_page.read()
    past_event_html = past_event_html_bytes.decode("utf-8")
    past_event_tables = pd.read_html(past_event_url)[0]

    body = past_event_html[past_event_html.index("<body "):]
    results = [row["W/L"] for i, row in past_event_tables.iterrows()]
    fight_http_strings = []
    for result in results:
        result = result.split(' ')[0]
        before_result = body[:body.index(result)]
        begin = before_result[before_result.rfind("http://"):]
        http = begin[:begin.find("\"")]
        fight_http_strings.append(http)
        body = body[body.index(result)+len(result):]
    return fight_http_strings

fight_http_strings = get_all_fights(all_event_http_strings[0])

In [156]:
print(len(fight_http_strings))

12


In [159]:
from tqdm import tqdm

In [160]:
all_fight_http_strings = []
for event_http_string in tqdm(all_event_http_strings):
    fight_http_strings = get_all_fights(event_http_string)
    all_fight_http_strings.extend(fight_http_strings)

100%|██████████| 537/537 [04:15<00:00,  2.10it/s]


In [161]:
print(len(all_fight_http_strings))

5839


In [181]:
def get_fighters(fight_table, all_fighters_table):
    for i, row in all_fighters_tables.iterrows():
        fighter_name = f'{row["First"]} {row["Last"]}'
        if fight_table["Fighter"][0].startswith(fighter_name):
            first_fighter = fighter_name
            second_fighter = fight_table["Fighter"][0][len(fighter_name)+1:]
            print(f"First fighter is {first_fighter}")
            print(f'Second fighter is {second_fighter}')
            break
    return first_fighter, second_fighter

In [184]:
def process_fight(fight_url):
    fight_tables = pd.read_html(fight_url)
    fight_table = fight_tables[1]
    fighter1, fighter2 = get_fighters(fight_tables[0], all_fighters_table)
    
    new_columns = ["Fighter 1", "Fighter 2"]
    for i, column in enumerate(fight_table):
        if column[0] == "Fighter":
            continue
        new_columns.append(f"Fighter 1 {column[0]}")
        new_columns.append(f"Fighter 2 {column[0]}")

    new_rows = []
    for i, row in fight_table.iterrows():
        new_row = []
        new_row.append(fighter1)
        new_row.append(fighter2)
        for column in fight_table:
            if column[0] == "Fighter":
                continue
            stat1, stat2 = parse_string(row[column[0]][0])
            new_row.append(stat1)
            new_row.append(stat2)
        new_rows.append(new_row)

    df = pd.DataFrame(new_rows, columns=new_columns)
    return df 

In [188]:
process_fight(all_fight_http_strings[8])

First fighter is Alex Oliveira
Second fighter is Shavkat Rakhmonov


,Fighter 1,Fighter 2,Fighter 1 KD,Fighter 2 KD,Fighter 1 Sig. str.,Fighter 2 Sig. str.,Fighter 1 Sig. str. %,Fighter 2 Sig. str. %,Fighter 1 Total str.,Fighter 2 Total str.,Fighter 1 Td %,Fighter 2 Td %,Fighter 1 Td %,Fighter 2 Td %,Fighter 1 Sub. att,Fighter 2 Sub. att,Fighter 1 Rev.,Fighter 2 Rev.,Fighter 1 Ctrl,Fighter 2 Ctrl
0,Alex Oliveira,Shavkat Rakhmonov,0,0,9 of 18,13 of 25,50%,52%,16 of 26,32 of 44,0 of 2,0 of 2,0 of 2,0 of 2,0,1,0,0,2:04,0:44
